In [29]:
pip install --upgrade --no-deps statsmodels

     ---------------------------------------- 9.1/9.1 MB 2.6 MB/s eta 0:00:00



In [31]:
pip install patsy

     -------------------------------------- 233.7/233.7 KB 1.1 MB/s eta 0:00:00



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2.extras

In [2]:
class PostgresConnection(object):
    def __init__(self):
        self.connection = psycopg2.connect(database = "ecomdb",
                                                  user = "postgres",
                                                  password = "sultanmahmud22",
                                                  host = "127.0.0.1",
                                                  port= "5432")
    def getConnection(self):
        print("Successfully connected to the database")
        return self.connection
con = PostgresConnection().getConnection()

Successfully connected to the database


In [3]:
cur = con.cursor()
select_stmt = "SELECT s.division, tim.month,  SUM(t.total_price) " \
"FROM star_schema.fact_table t  " \
"JOIN star_schema.store_dim s on s.store_key = t.store_key " \
"JOIN star_schema.time_dim tim on tim.time_key = t.time_key " \
"GROUP BY CUBE(s.division, tim.month) " 
cur.execute(select_stmt)
records = cur.fetchall()
records

df = pd.DataFrame(list(records), columns = ['Division','Month', 'Sales'])
df = df.dropna()
df

,Division,Month,Sales
1,DHAKA,11.0,3343397.00
2,SYLHET,1.0,485369.75
3,CHITTAGONG,8.0,1667686.75
4,CHITTAGONG,11.0,1640113.75
5,RAJSHAHI,3.0,1002924.25
...,...,...,...
80,RAJSHAHI,10.0,1034440.50
81,BARISAL,2.0,582863.00
82,SYLHET,5.0,479031.75
83,CHITTAGONG,9.0,1627565.25


In [4]:
df = df.iloc[:, :].values

In [5]:
from sklearn.preprocessing import LabelEncoder
division=LabelEncoder()
df[:,0]=division.fit_transform(df[:,0])

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encode',OneHotEncoder(),[0])],remainder='passthrough')
y=ct.fit_transform(df)
y=pd.DataFrame(y)
y

,0,1,2,3,4,5,6,7,8
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11.0,3343397.00
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,485369.75
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,8.0,1667686.75
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,11.0,1640113.75
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,1002924.25
...,...,...,...,...,...,...,...,...,...
79,0.0,0.0,0.0,0.0,1.0,0.0,0.0,10.0,1034440.50
80,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,582863.00
81,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,479031.75
82,0.0,1.0,0.0,0.0,0.0,0.0,0.0,9.0,1627565.25


In [19]:
X = y.iloc[:, :8].values;
X = X[:, 1:]

In [20]:
Y = y.iloc[:, -1].values;

In [21]:
# Splittting the dataset into the Training set and Test set

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0);


In [22]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)


In [23]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, Y_train)

LinearRegression()

In [24]:
y_pred = regressor.predict(X_test)
y_pred

array([ 939038.37351546,  705036.88467937, 3404554.76935874,
        707329.00819483,  934454.12648454,  458421.44046908,
       1648119.69673401,  463005.6875    , 1016299.40504216,
       3395386.27529691,  474466.30507729, 1657288.19079584,
        622933.91502372,  950498.99109275,  625226.03853918,
        936746.25      ,  469882.05804637])

In [25]:
Y_test

array([Decimal('977201.75'), Decimal('719549.00'), Decimal('3483408.25'),
       Decimal('703469.00'), Decimal('966610.25'), Decimal('450152.25'),
       Decimal('1667686.75'), Decimal('437286.00'), Decimal('988666.25'),
       Decimal('3322764.50'), Decimal('451435.50'), Decimal('1665138.00'),
       Decimal('640123.00'), Decimal('952049.75'), Decimal('611915.50'),
       Decimal('921778.75'), Decimal('469315.00')], dtype=object)

In [40]:
# import statsmodels.api as sm
# X=np.append(arr=np.ones((84,1)).astype(int), values = X, axis = 1)
# X_opt = X[:, [0,1,2,3,4,5,6,7]]
# regressor_OLS =  sm.OLS(endog = Y, exog = X_opt).fit()
# regressor_OLS.summary()

In [18]:
# # Visualising the Training set results
# plt.scatter(X_train,Y_train, color = 'red')
# plt.plot(X_train,regressor.predict(X_train), color='blue')
# # plt.title('Salary vs Experience (Training set)')
# # plt.xlabel('Years of Experience')
# # plt.ylabel('Salary')
# plt.show()

### SVR

In [26]:
# sc_X = StandardScaler()
# sc_Y = StandardScaler()

# X = sc_X.fit_transform(X)
# Y = sc_Y.fit_transform(Y)

In [23]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf') # kernel = linear, poly, rbf, sigmoid, precomputed
regressor.fit(X_train, Y_train)

SVR()

In [24]:
y_pred = regressor.predict(X_test)
y_pred

array([943267.56264668, 943261.18311315, 943275.33955722, 943261.43858596,
       943267.72309068, 943262.26146438, 943276.05164443, 943262.10301451,
       943275.27270987, 943276.20780988, 943263.14720316, 943274.1938841 ,
       943261.19612866, 943267.42008569, 943261.05922857, 943267.63817923,
       943262.5112933 ])

In [14]:
# from sklearn.preprocessing import OneHotEncoder
# oe_style = OneHotEncoder()
# oe_results = oe_style.fit_transform(df[["Division"]])
# pd.DataFrame(oe_results.toarray(), columns=oe_style.categories_).head()

In [15]:
# #df_a.join(df_b, on='mukey', how='left', lsuffix='_left', rsuffix='_right')
# a = pd.DataFrame(oe_results.toarray(), columns=oe_style.categories_)
# df = pd.concat(a)
# df


In [16]:
# df['Division'] = df['Division'].astype('category')
# df["Division"] = df["Division"].cat.codes
# # df.head()
# df = pd.get_dummies(df, columns=["Division"], prefix=['Division']).head()

In [17]:
# df['Division'] = df['Division'].astype('category')
# df["Division"] = df["Division"].cat.codes
# df.head()
# pd.get_dummies(df, columns=["Division"]).head()

In [18]:
# # Splittting the dataset into the Training set and Test set

# from sklearn.model_selection import train_test_split
# X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0);
# print(X_train)
# print(Y_train)
# print(X_test)
# print(Y_test)

In [19]:
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
# from sklearn import preprocessing
# # LabelEncoder_X = LabelEncoder()
# # X[:, 0] = LabelEncoder_X.fit_transform(X[:, 0])
# # # # OneHotEncoder = OneHotEncoder(categories_[0])
# # # # X = OneHotEncoder.fit_transform(X).toarray()
# # # OneHotEncoder = preprocessing.OneHotEncoder()

# # # 2. FIT
# # OneHotEncoder.fit(X)

# # # 3. Transform
# # X = OneHotEncoder.transform(X).toarray()
# # X.shape
# ohe = OneHotEncoder()
# ohe.fit_transform(df[['Division']])
# # df['Division'].head()
# # ohe.categories_

In [20]:
# df['Division']
# ohe.categories_

In [21]:
# pd.get_dummies(df['Division'])

In [22]:
# column_trans_form = make_column_transformer(
# (ohe, ['Division']), 'mMonth', 'Quantity'
# )

In [23]:
# # Feature Scaling

# from sklearn.preprocessing import StandardScaler
# sc_X = StandardScaler()
# X_train = sc_X.fit_transform(X_train)
# X_test = sc_X.transform(X_test)
# print(X_train)

In [24]:
# from sklearn.linear_model import LinearRegression
# regressor = LinearRegression()
# regressor.fit(X_train, Y_train)

In [25]:
# y_pred = regressor.predict(X_test)
# y_pred